In [2]:
from transformers import AutoTokenizer
import pandas as pd
from transformer import Transformer, Transformer_Trainer, Transformer_Dataset
import torch
from torch.utils.data import DataLoader


c:\Users\1027d\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
checkpoint = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [4]:
batch_size = 4

df = pd.read_csv("../Data_Process/mbti_1.csv")

token_mbti = {"[PAD]" : 0, "[CLS]" : 1, "[SOS]" : 2, "I" : 3, "E" : 4, "N" : 5, "S" : 6, "F" : 7, "T" : 8, "J" : 9, "P" : 10}

for i in range(len(df["type"])):
    df["type"][i] = [1, token_mbti[df["type"][i][0]], token_mbti[df["type"][i][1]], token_mbti[df["type"][i][2]], token_mbti[df["type"][i][3]], 2]


data = [(df["posts"][i], df["type"][i]) for i in range(len(df["posts"]))]




C:\Users\1027d\AppData\Local\Temp\ipykernel_25000\1981573403.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["type"][i] = [1, token_mbti[df["type"][i][0]], token_mbti[df["type"][i][1]], token_mbti[df["type"][i][2]], token_mbti[df["typ

In [5]:
dataset = Transformer_Dataset(data, tokenizer, seq_len=128)


In [6]:
test_loader = DataLoader(
    dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [7]:

src_vocab_size = tokenizer.vocab_size
tgt_vocab_size = 11 # [PAD], [SOS], [CLS], I, E, N, S, F, T, J, P
d_model = 128
num_heads = 8
num_layers = 6
d_ff = 512
max_seq_length = 128
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout, device = "cuda")

transformer.load_state_dict(torch.load("../Trained_Model/transformer_1"))


transformer.cuda()



Transformer(
  (encoder_embedding): Embedding(28996, 128)
  (decoder_embedding): Embedding(11, 128)
  (positional_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
      )
      (feed_forward): FeedForward(
        (fc1): Linear(in_features=128, out_features=512, bias=True)
        (fc2): Linear(in_features=512, out_features=128, bias=True)
        (activation): ReLU()
      )
      (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (decoder_layers): ModuleList(
    (0-5): 6 x DecoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
      )
      (cross_attn): MultiheadAttention(

In [8]:
device = "cuda"

In [9]:
total_correct_IE = 0
total_correct_NS = 0
total_correct_FT = 0
total_correct_JP = 0

transformer.eval()
for i in test_loader:
    
    i = {key: value.to(device) for key, value in i.items()}
    res = transformer.prediction(i["Encoder_input"], i["Decoder_input"])

    total_correct_IE += res[:, 0].eq(i["Decoder_output"][:, 0]).sum().item()
    total_correct_NS += res[:, 1].eq(i["Decoder_output"][:, 1]).sum().item()
    total_correct_FT += res[:, 2].eq(i["Decoder_output"][:, 2]).sum().item()
    total_correct_JP += res[:, 3].eq(i["Decoder_output"][:, 3]).sum().item()

    print("expected: ", i["Decoder_output"])
    print("out: ", res)

expected:  tensor([[ 3,  5,  7,  9],
        [ 4,  5,  8, 10],
        [ 3,  5,  8, 10],
        [ 3,  5,  8,  9]], device='cuda:0')
out:  tensor([[ 3,  5,  7, 10],
        [ 3,  5,  7, 10],
        [ 3,  5,  7, 10],
        [ 3,  5,  7, 10]], device='cuda:0')
expected:  tensor([[4, 5, 8, 9],
        [3, 5, 8, 9],
        [3, 5, 7, 9],
        [3, 5, 8, 9]], device='cuda:0')
out:  tensor([[ 3,  5,  7, 10],
        [ 3,  5,  7, 10],
        [ 3,  5,  7, 10],
        [ 3,  5,  7, 10]], device='cuda:0')
expected:  tensor([[ 3,  5,  7,  9],
        [ 3,  5,  8, 10],
        [ 3,  5,  7,  9],
        [ 4,  5,  7,  9]], device='cuda:0')
out:  tensor([[ 3,  5,  7, 10],
        [ 3,  5,  7, 10],
        [ 3,  5,  7, 10],
        [ 3,  5,  7, 10]], device='cuda:0')
expected:  tensor([[ 3,  5,  7,  9],
        [ 3,  5,  8,  9],
        [ 3,  5,  8, 10],
        [ 3,  5,  8, 10]], device='cuda:0')
out:  tensor([[ 3,  5,  7, 10],
        [ 3,  5,  7, 10],
        [ 3,  5,  7, 10],
        [ 3,  5,

In [11]:
total_correct_IE/len(df)

0.7695677233429394

In [13]:
total_correct_NS/len(df)

0.8620172910662824

In [12]:
total_correct_FT/len(df)

0.5571181556195965

In [14]:
total_correct_JP/len(df)

0.604149855907781